In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install geocoder
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [2]:
import requests
import geocoder
import pandas as pd
import numpy as np
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

<div style="background-color:rgba(204, 255, 204, 0.470588); text-align:center; vertical-align: middle; padding:20px 0; margin-top:30px">
    <h1>Scrape Canada Postal Codes wikitable</h>
</div>

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url).text
soup = BeautifulSoup(res,'lxml')

In [4]:
postcode, borough, neighbourhood = [],[],[]
for items in soup.find('table', class_='wikitable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        postcode.append(data[0].text)
        borough.append(data[1].text)
        neighbourhood.append(data[2].text)
    except IndexError:pass
    
data = {'PostalCode':postcode, 'Borough':borough, 'Neighborhood':neighbourhood} 
# Create DataFrame 
df = pd.DataFrame(data) 
print(df.shape)
df.head()

(288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#### remove rows with unassigned borough

In [5]:
df = df[~df.Borough.str.contains("Not assigned")]
# strip the newline sign in neighbourhood 
df = df.replace('\n','', regex=True)
print(df.shape)
df.head()

(211, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


check how many unique post code in the table

In [6]:
df['PostalCode'].nunique()

103

#### combine neighborhood into the same colum when they share the same postal code

In [7]:
df['Neighborhood'] = df.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df = df.drop_duplicates()
print(df.shape)
df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned


#### Assign borough name to 'Not assigned' neighborhood

In [8]:
df['Neighborhood'][df['Neighborhood'] == 'Not assigned'] = df['Borough']
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


<div style="background-color:rgba(204, 255, 204, 0.470588); text-align:center; vertical-align: middle; padding:20px 0; margin-top:30px">
    <h1>Get Geographical Coordinates (latitude and longitude) for Each Post Code</h>
</div>

```
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude)
print(longitude)
```

above demo code runs forever without response, so try to use coordinates provided by the course and combine to existing df

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [10]:
coordinates_df = pd.read_csv("Geospatial_Coordinates.csv", delimiter=",")
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine coordinates and wikitable based on common postal codes

In [11]:
cmb = pd.merge(left=df,right=coordinates_df, left_on='PostalCode', right_on='Postal Code')
cmb.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [12]:
cmb = cmb.drop(columns=['Postal Code'])
print(cmb.shape)
cmb.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


<div style="background-color:rgba(204, 255, 204, 0.470588); text-align:center; vertical-align: middle; padding:20px 0; margin-top:30px">
    <h1>Segmenting and Clustering Neighborhoods in Toronto</h>
</div>

#### Extract Boroughs Containing the Word 'Toronto'

In [13]:
toronto_data = cmb[cmb['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


#### Define Foursquare Credentials and Version

In [14]:
CLIENT_ID = 'XVCRNREVOI2ADSSZ2PPGMYFA3AVHLUZGESTC0CQWYJJILP3Z' # your Foursquare ID
CLIENT_SECRET = '2UYHRXV5X0SUDPPEICOGJRYDHC1MVMEITRVYHTWDQ25YKGIW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Explore Neighborhoods in Toronto

Borrow functions from previous lab

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [17]:
print(toronto_venues.shape)
toronto_venues.head(3)

(1710, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center


check how many venues were returned for each neighborhood

In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",45,45,45,45,45,45
Central Bay Street,86,86,86,86,86,86
"Chinatown,Grange Park,Kensington Market",93,93,93,93,93,93
Christie,17,17,17,17,17,17


how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 234 uniques categories.


#### Analyze Each Neighborhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head(3)

(1710, 234)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head(3)

(38, 234)


,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.00


**Return each neighborhood along with the top N most common venues**

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(3)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Bakery,Steakhouse,Sushi Restaurant,Asian Restaurant,Burger Joint,American Restaurant
1,Berczy Park,Coffee Shop,Café,Cheese Shop,Farmers Market,Bakery,Steakhouse,Seafood Restaurant,Italian Restaurant,Beer Bar,Cocktail Bar
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Grocery Store,Furniture / Home Store,Burrito Place,Sandwich Place,Restaurant,Stadium,Italian Restaurant


In [24]:
neighborhoods_venues_sorted.shape

(38, 11)

#### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Café,Park,Bakery,Pub,Mexican Restaurant,Restaurant,Theater,Breakfast Spot,Brewery
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bakery,Pizza Place,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Hotel,Restaurant,Clothing Store,Breakfast Spot,Italian Restaurant,Bakery,Gastropub,Cosmetics Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Café,Cheese Shop,Farmers Market,Bakery,Steakhouse,Seafood Restaurant,Italian Restaurant,Beer Bar,Cocktail Bar


visualize the resulting clusters

In [27]:
# Toronto city geolocation
latitude = 43.682339
longitude = -79.299514

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

**Cluster 1**

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Café,Park,Bakery,Pub,Mexican Restaurant,Restaurant,Theater,Breakfast Spot,Brewery
1,Downtown Toronto,0,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bakery,Pizza Place,Japanese Restaurant,Italian Restaurant,Sporting Goods Shop,Ramen Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Clothing Store,Breakfast Spot,Italian Restaurant,Bakery,Gastropub,Cosmetics Shop
4,Downtown Toronto,0,Coffee Shop,Café,Cheese Shop,Farmers Market,Bakery,Steakhouse,Seafood Restaurant,Italian Restaurant,Beer Bar,Cocktail Bar
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Ice Cream Shop,Burger Joint,Sandwich Place,Chinese Restaurant,Japanese Restaurant,Gym / Fitness Center,Bakery
6,Downtown Toronto,0,Grocery Store,Café,Park,Nightclub,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant,Candy Store
7,Downtown Toronto,0,Coffee Shop,Café,Bar,Thai Restaurant,Bakery,Steakhouse,Sushi Restaurant,Asian Restaurant,Burger Joint,American Restaurant
8,West Toronto,0,Supermarket,Bakery,Pharmacy,Bank,Bar,Café,Music Venue,Furniture / Home Store,Middle Eastern Restaurant,Pizza Place
9,Downtown Toronto,0,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Italian Restaurant,Fried Chicken Joint,Brewery,Sporting Goods Shop
10,West Toronto,0,Bar,Coffee Shop,Restaurant,Men's Store,Asian Restaurant,French Restaurant,Café,Vietnamese Restaurant,Pizza Place,New American Restaurant


**Cluster 2**

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
32,Downtown Toronto,1,Park,Playground,Trail,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


**Cluster 3**

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,2,Gym / Fitness Center,Park,Swim School,Bus Line,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


**Cluster 4**

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Garden,Ice Cream Shop,Dessert Shop,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


**Cluster 5**

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Health Food Store,Trail,Pub,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
